## ch 1. python requests를 이용한 크롤러 개발

### 1. 필요 라이브러리
이제 본격적으로 크롤러를 개발해보겠습니다. 웹 크롤러를 만들기 위해서는 먼저 http client가 있어야 합니다. 지금까지는 웹 브라우저를 클라이언트로 사용하였는데, python의 reuests라는 라이브러리를 사용하면 쉽게 HTTP 요청을 보낼 수 있습니다. 그리고 응답으로 수신한 HTML 문서를 쉽게 파싱하기 위해서 BeautifulSoup이라는 라이브러리를 이요하겠습니다.

- requests
- BeautifualSoup

In [19]:
!pip install requests
!pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
# beautifulsoup = 기본적인 html 문서를 beautifulsoup 객체로 만들면 find 같은 함수 사용 가능 그래서 title이나 id 등 태그 활용해 원하는 정보만 파싱하기 위한 라이브러리
# 웹서비스가 어떻게 동작하는지 구조 파악하기
# 개발자 도구 f12  -> network(서버와 주고받은 데이터 확인 가능)
# 개발자 도구 f12  -> network -> fetch/XHR = 원하는 데이터 요청하는 API  -> copy curl(bash) -> vscode에서 git bash터미널에 붙여넣기 하면 데이터 받아올 수 있음
# 웹서비스 = 서버로부터 데이터를 주고 받는다 !!

In [76]:
import requests
from bs4 import BeautifulSoup  #-> 뷰티풀숩은 클래스임 bs4에서 Beautifulsoup 클래스 가져오겠다

### 2. news list backend API에 HTTP 요청 보내기
뉴스 기사 목록을 내려주는 네이버 스포츠 백엔드 API에 2023년 5월 10일 뉴스 기사 목록을 요청을 보낸 뒤, 응답을 json 객체로 바꿔보겠습니다. json은 python에서는 dict 데이터 타입으로 표현됩니다.

In [13]:
import time

start = time.time()
sample_url = "https://sports.news.naver.com/kbo/news/list?isphoto=N&page=2"
response = requests.get(sample_url)
response_dict = response.json() 

print("elapsed", time.time() - start)

elapsed 1.0464468002319336


In [8]:
sample_url = "https://sports.news.naver.com/kbo/news/list?isphoto=N&page=2"
response = requests.get(sample_url)

In [9]:
response_dict = response.json()    #json -> dictionary로 받기 

In [10]:
response_dict

{'list': [{'oid': '031',
   'aid': '0000795412',
   'officeName': '아이뉴스24',
   'title': '키움 김혜성, 2023 KBO 페어플레이상 선정',
   'subContent': '한국야구위원회(KBO)가 2023 KBO리그 페어플레이이상 수상자로 키움 히어로즈 김혜성(내야수)를 선정했다고 11일 밝혔다. 페어플레이상은 지난달(11월)...',
   'thumbnail': 'http://imgnews.naver.net/image/thumb154/031/2023/12/11/795412.jpg',
   'datetime': '2023.12.11 10:15',
   'url': None,
   'sectionName': 'KBO리그',
   'type': 'PHOTO',
   'totalCount': 50},
  {'oid': '001',
   'aid': '0014382582',
   'officeName': '연합뉴스',
   'title': '키움 김혜성, 2023 페어플레이상 수상자로 선정',
   'subContent': 'KBO 시상식에서 수비상 받은 김혜성[연합뉴스 자료사진] (서울=연합뉴스) 김경윤 기자 = 프로야구 키움 히어로즈의 주전 내야수 김혜성(24)이 2023 K...',
   'thumbnail': 'http://imgnews.naver.net/image/thumb154/001/2023/12/11/14382582.jpg',
   'datetime': '2023.12.11 10:13',
   'url': None,
   'sectionName': 'KBO리그',
   'type': 'PHOTO',
   'totalCount': 63},
  {'oid': '640',
   'aid': '0000047077',
   'officeName': '코리아중앙데일리',
   'title': 'Hanwha Eagles re-sign Felix Pena on one-year, $1.05 mill

### 3. 반목문을 돌면서 모든 페이지의 뉴스 기사 oid, aid 수집하기
totalPages 값을 통해서 2023년 5월 10일 야구 뉴스 기사가 총 25 페이지 있다는 것을 알게되었습니다. 우리가 원하는 것은 특정일에 나온 뉴스 기사들의 oid와 aid들입니다. 한번 for문을 돌면서 모든 페이지를 요청하고, oid와 aid를 수집해보겠습니다. 

전체 진행 상황을 모니터링 하기 위해서 tqdm 라이브러리를 사용해보겠습니다.  

### 4. 뉴스 기사 페이지를 요청한 뒤, 기사 제목과 본문 파싱하기 
기사 상세 페이지를 하나만 요청한 뒤, BeautifulSoup을 이용해 파싱해보겠습니다. BeautifulSoup 객체를 만들 때 뒤에 붙여주는 "lxml"은 BeautifulSoup에 내장된 HTML 파서 중 가장 많이 사용되는 것입니다.

In [32]:
news_list = response_dict["list"]
for news_item in news_list:
    oid = news_item["oid"]
    aid = news_item["aid"]
    url = f"https://sports.naver.com/news?oid={oid}&aid={aid}"
    #print(url)
    news_item_response = requests.get(url)
    news_item_html = news_item_response.text
    # print(news_item_html)
    news_item_soup = BeautifulSoup(news_item_html)
    
    title = news_item_soup.find("title").text
     # 본문 추출 -> 상세페이지에서 우클릭 - 검사 클릭릭
    content = news_item_soup.find("div", id ="newsEndContents").text
    
    break

In [35]:
# p태그 안 데이터 제거하기
news_list = response_dict["list"]
for news_item in news_list:
    oid = news_item["oid"]
    aid = news_item["aid"]
    url = f"https://sports.naver.com/news?oid={oid}&aid={aid}"
    news_item_response = requests.get(url)
    news_item_html = news_item_response.text
    news_item_soup = BeautifulSoup(news_item_html)
    title = news_item_soup.find("title").text
    
    content = news_item_soup.find("div", id ="newsEndContents")
    p_tags = content.find_all("p")
    for p_tag in p_tags:
        p_tag.decompose()
    print(content.text)
    break


11일 골든글러브 시상식서 수상키움 김혜성이 27일 서울 중구 웨스틴조선호텔에서 열린 2023 신한은행 SOL KBO 시상식에서 2루수 부문 수비상을 수상한 후 수상소감을 전하고 있다. 2023.11.27/뉴스1 ⓒ News1 김성진 기자(서울=뉴스1) 서장원 기자 = 키움 히어로즈 내야수 김혜성이 2023 KBO 페어플레이상 수상자로 선정됐다.KBO는 11일 "지난달 23일 상벌위원회를 열고 2023 KBO 페어플레이상 수상자로 키움 김혜성을 선정했다"고 밝혔다.2001년에 제정된 페어플레이상은 KBO 정규시즌 기간 스포츠 정신에 입각한 진지한 태도와 판정 승복으로 타의모범이 돼 KBO리그 이미지 향상에 기여한 선수에게 시상된다.김혜성은 개인 첫 번째 페어플레이상을 받게 됐고, 소속팀 키움은 지난해 이지영에 이어 2년 연속 수상자를 배출하게 됐다.시상은 11일 오후 5시부터 개최되는 KBO 골든글러브 시상식에서 진행된다.
						
						
						
						
						









서장원 기자


구독





예
아니오

닫기






구독자
-


응원수
-








GG 수상 유력한 페디, 양의지 넘어 역대 최고 득표율 정조준


프로농구 흥행몰이 가속화…관중수·입장 수익 모두 증대













기사 섹션 분류 가이드

기사 섹션 분류 안내

오분류 제보하기
가이드 닫기






K팝·K트롯 팬들의 놀이터, 스타1픽


세상에 이런 일이...[사건의 재구성]






기사 본문 외에도 기자 명이나 언론사 링크 등의 불필요한 정보들이 포함되어 있다. 예를 들어 div id="newsEndContents" 태그 안에 포함된 p, div, span, em 태그들은 모두 불필요한 텍스트들을 가지고 있다. 이를 BeautifulSoup를 이용해서 제거한다. 

In [50]:
# 크롤러 코드 함수화하기

In [51]:
# 특정 태그 안 데이터 제거하는 함수
def remove_tag(parent_soup, target):
    p_tags = parent_soup.find_all(target)
    for p_tag in p_tags:
        p_tag.decompose()

In [52]:
# content에서 불필요한 데이터 제거해주는 함수
def parse_content(news_item_soup):
    content = news_item_soup.find("div", id ="newsEndContents")
    remove_tag(content, "p")
    remove_tag(content, "div")
    remove_tag(content, "em")
    remove_tag(content, "span")
    content_text = content.text.strip()    # 공백 제거
    return content_text

In [53]:
# 기사 제목만 뽑아주는 함수
def parse_title(news_item_soup):
    title = news_item_soup.find("title").text
    return title

In [54]:
news_list = response_dict["list"]
for news_item in news_list:
    oid = news_item["oid"]
    aid = news_item["aid"]
    url = f"https://sports.naver.com/news?oid={oid}&aid={aid}"
    news_item_response = requests.get(url)
    news_item_html = news_item_response.text
    news_item_soup = BeautifulSoup(news_item_html)
    title = parse_title(news_item_soup)
    content = parse_content(news_item_soup)
    print(title, content)
    break

김혜성, KBO 페어플레이 수상자 선정…키움 2년 연속 배출 11일 골든글러브 시상식서 수상(서울=뉴스1) 서장원 기자 = 키움 히어로즈 내야수 김혜성이 2023 KBO 페어플레이상 수상자로 선정됐다.KBO는 11일 "지난달 23일 상벌위원회를 열고 2023 KBO 페어플레이상 수상자로 키움 김혜성을 선정했다"고 밝혔다.2001년에 제정된 페어플레이상은 KBO 정규시즌 기간 스포츠 정신에 입각한 진지한 태도와 판정 승복으로 타의모범이 돼 KBO리그 이미지 향상에 기여한 선수에게 시상된다.김혜성은 개인 첫 번째 페어플레이상을 받게 됐고, 소속팀 키움은 지난해 이지영에 이어 2년 연속 수상자를 배출하게 됐다.시상은 11일 오후 5시부터 개최되는 KBO 골든글러브 시상식에서 진행된다.


In [ ]:
# 12월 9일 야구 뉴스 1~6페이지 가져오기

In [73]:
url_template = "https://sports.naver.com/kbaseball/news/index?isphoto=N&sort=latest&isPhoto=N&date=20231209&page={target_page}"
for i in range(6):
    url = url_template.format(target_page=i+1)
    print(url)

https://sports.naver.com/kbaseball/news/index?isphoto=N&sort=latest&isPhoto=N&date=20231209&page=1
https://sports.naver.com/kbaseball/news/index?isphoto=N&sort=latest&isPhoto=N&date=20231209&page=2
https://sports.naver.com/kbaseball/news/index?isphoto=N&sort=latest&isPhoto=N&date=20231209&page=3
https://sports.naver.com/kbaseball/news/index?isphoto=N&sort=latest&isPhoto=N&date=20231209&page=4
https://sports.naver.com/kbaseball/news/index?isphoto=N&sort=latest&isPhoto=N&date=20231209&page=5
https://sports.naver.com/kbaseball/news/index?isphoto=N&sort=latest&isPhoto=N&date=20231209&page=6


In [94]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable


### 5. 코드 정리 및 CSV 파일에 데이터 쓰기
지금까지 네이버 스포츠에서 특정 요일에 야구 기사 목록을 가져온 뒤, 각각의 기사의 제목과 본문을 수집하는 크롤러를 개발해보았다. 코드를 정리해보면 아래와 같다.

In [98]:
import csv
from tqdm.notebook import tqdm    # 반복문 돌 때 진행상황 표시해줌

url_template = "https://sports.naver.com/kbaseball/news/list?date=20231209&isphoto=N&page={target_page}&pageSize=20"
with open("./data/baseball_news.csv", "w", encoding="utf-8-sig", newline="") as fw:
    writer = csv.writer(fw)
    writer.writerow(["url", "title", "content"])
    for i in range(6):
        url = url_template.format(target_page=i+1)
        response = requests.get(url)
        response_dict = response.json()
        news_list = response_dict["list"]
        for news_item in tqdm(news_list, total=len(news_list)):
            oid = news_item["oid"]
            aid = news_item["aid"]
            url = f"https://sports.naver.com/news?oid={oid}&aid={aid}"
            news_item_response = requests.get(url)
            news_item_html = news_item_response.text
            news_item_soup = BeautifulSoup(news_item_html)
            title = parse_title(news_item_soup)
            content = parse_content(news_item_soup)
            writer.writerow([news_url, title, content])


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]